In [33]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pandas as pd
import numpy as np
sys.path.insert(0, r'C:\workspace\characterization')
from matplotlib import pyplot as plt
import fivecentplots as fcp
from characterization_ams.stats_engine import pixelwise_stats as ps
from characterization_ams.stats_engine import stats
from characterization_ams.standard_tests import ptc
from characterization_ams.unittests import image_generator
from characterization_ams.utilities import image
import pdb
import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
gen_images = image_generator.gen_images
rows=5000
cols=5000

# fpn
rfpn=15
pfpn=5
cfpn=1
tot_f = np.sqrt(rfpn**2 + pfpn**2 + cfpn**2)

# temp noise
ctn = 15
rtn = 12
ptn = 20
tot_t = np.sqrt(ctn**2 + rtn**2 + ptn**2)
n_images=5
pedestal = 168
imgs = gen_images(cfpn=cfpn,
                  ptn=ptn,
                  rtn=rtn,
                  ctn=ctn,
                  rfpn=rfpn,
                  pfpn=pfpn,
                  rows=rows,
                  cols=cols,
                  pedestal=pedestal,
                  n_images=n_images)

In [35]:
imgs[0].shape
imgs_ = np.array(imgs)

## Speed Test

In [36]:
img = imgs[0]

### Spatial Variance

#### Masked Array

In [37]:
mask = np.zeros((img.shape[0], img.shape[1]))
mask[1::4] = 1

In [38]:
@measure
def mavar(img, mask=np.ma.nomask):
    img = np.ma.masked_array(img, mask=mask)
    var = np.ma.var(img)
    return var

##### No mask

In [39]:
mavar(img, mask=np.ma.nomask)

Total execution time: 285 ms


1025.9228198879086

##### Mask

In [40]:
mavar(img, mask=mask)

Total execution time: 946 ms


1031.6418947044274

#### Normal Array

In [41]:
@measure
def var(img):
    var = np.var(img)
    return var

In [42]:
var(img)

Total execution time: 241 ms


1025.9228198879086

## Temporal Noise

### Masked Array

In [43]:
@measure
def temp_mavar(imgs, mask=np.ma.nomask):
    imgs = np.ma.masked_array(imgs, mask=mask)
    temp_var = np.ma.mean(np.ma.var(imgs, axis=0))
    return temp_var

#### No Mask

In [44]:
temp_mavar(imgs_, mask=np.ma.nomask)

Total execution time: 2252 ms


615.694975251597

#### Mask

In [45]:
mask = np.zeros((imgs_.shape[0], imgs_.shape[1],imgs_.shape[2]))
mask[1::4] = 1

In [46]:
temp_mavar(imgs_, mask=mask)

Total execution time: 7954 ms


579.5695464456087

#### Normal Array

In [47]:
@measure
def temp_var(imgs):
    var = np.var(imgs, axis=0).mean()
    return var

In [48]:
temp_var(imgs_)

Total execution time: 1435 ms


615.694975251597

## helper Functions

In [49]:
from functools import wraps
from time import time
def measure(func):
    @wraps(func)
    def _time_it(*args, **kwargs):
        start = int(round(time() * 1000))
        try:
            return func(*args, **kwargs)
        finally:
            end_ = int(round(time() * 1000)) - start
            print(f"Total execution time: {end_ if end_ > 0 else 0} ms")
    return _time_it
